In [1]:
import math
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.optimize import leastsq
from sklearn.linear_model import LinearRegression
from scipy import sparse
import numpy as np

## 一.品牌匹配

In [5]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['body','energy'],axis=1)
car_autohome_all = car_autohome_all.loc[:, ['brand_name','brand_slug','model_name','model_slug','detail_name','detail_slug','online_year','price_bn']]
car_autohome_all = car_autohome_all.sort_values(by=['brand_name','model_name','online_year','price_bn'])

# 汽车之家品牌
car_autohome_brand = car_autohome_all.loc[:, ['brand_name','brand_slug']].drop_duplicates(['brand_slug']).rename(columns={'brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家品牌数量:', len(car_autohome_brand))

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id','name']].rename(columns={'name':'brand_name'}).reset_index(drop=True)
print('力洋品牌数量:', len(liyang_brand))
brand = liyang_brand.merge(car_autohome_brand, how='left', on=['brand_name'])
brand = brand.sort_values(by=['brand_name'])
car_autohome_brand = car_autohome_brand.sort_values(by=['brand_name'])
car_autohome_brand.to_csv('../tmp/train/match_car_autohome_brand.csv', index=False)
brand.to_csv('../tmp/train/final_brand.csv', index=False)
brand.info()
print('请人工匹配未匹配上品牌.')

汽车之家品牌数量: 196
力洋品牌数量: 156
<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 114 to 111
Data columns (total 3 columns):
id                       156 non-null int64
brand_name               156 non-null object
car_autohome_brand_id    112 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.9+ KB
请人工匹配未匹配上品牌.


## 二.车系匹配

In [28]:
brand = pd.read_csv('../tmp/train/final_brand.csv')

open_category = pd.read_csv('../tmp/train/open_category.csv')
open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv')
open_category_brand = open_category.loc[(open_category['parent'].isnull()), ['id','slug']].rename(columns={'slug':'parent'}).reset_index(drop=True)
brand = open_category_brand.merge(brand,how='left',on=['id'])
model = open_category.loc[(open_category['parent'].notnull()), ['id','name','slug','parent']].rename(columns={'name':'model_name',
                         'slug':'model_slug','id':'model_id'}).reset_index(drop=True)
model = model.merge(brand, how='left', on=['parent']).rename(columns={'id':'brand_id'})
model = model.loc[:, ['brand_id','model_id','brand_name','model_name','model_slug','car_autohome_brand_id']]
print('力洋车系数量:', len(model))

# 汽车之家车系
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
print('汽车之家车系数量:', len(car_autohome_model))

model = model.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
model.loc[(model['car_autohome_brand_id'].isnull()), 'car_autohome_brand_id'] = -1
model.loc[(model['car_autohome_brand_id'] == -1), 'car_autohome_model_id'] = -1
model = model.sort_values(by=['brand_name','model_name'])
model.to_csv('../tmp/train/final_model.csv', index=False)

temp = car_autohome_all.loc[:, ['model_slug','brand_name','model_name']].drop_duplicates(['model_slug'])                                                                                                                            
temp.to_csv('../tmp/train/match_car_autohome_model.csv', index=False)
model.info()
# model.head()
# model = model.merge(brand, how='left', on=['parent']).rename(columns={'parent':'brand_slug'})

力洋车系数量: 1765
汽车之家车系数量: 1826
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1765 entries, 890 to 539
Data columns (total 7 columns):
brand_id                 1765 non-null int64
model_id                 1765 non-null int64
brand_name               1765 non-null object
model_name               1765 non-null object
model_slug               1765 non-null object
car_autohome_brand_id    1765 non-null float64
car_autohome_model_id    608 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 110.3+ KB


In [47]:
def process_model_name(df):
    text = df['model_name']
    text = text.replace(df['brand_name'],'')
    text = text.replace(' ', '')
    text = text.lower()
    return text

model = pd.read_csv('../tmp/train/final_model.csv')
part1 = model.loc[(model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
part2 = model.loc[(model['car_autohome_model_id'].isnull()), :].reset_index(drop=True)
part2['model_name'] = part2.apply(process_model_name, axis=1)
part2 = part2.drop(['car_autohome_model_id'], axis=1)
car_autohome_model = car_autohome_all.loc[:, ['brand_slug','brand_name','model_slug','model_name']].drop_duplicates(['model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model = car_autohome_model.drop(['brand_name'], axis=1)
part2 = part2.merge(car_autohome_model, how='left', on=['car_autohome_brand_id','model_name'])
final = part1.append(part2, sort=False)
final = final.sort_values(by=['brand_name','model_name'])
final.to_csv('../tmp/train/final_model.csv', index=False)
part2.head()

,brand_id,model_id,brand_name,model_name,model_slug,car_autohome_brand_id,car_autohome_model_id
0,1851,1881,Pagani,zonda,bseries_964,61,NaN
1,1346,1203,一汽,佳宝,jiabao,110,NaN
2,1346,1295,一汽,夏利2000,xiali2000,110,NaN
3,1346,1291,一汽,夏利a+,xiali-ajia,110,NaN
4,1346,1292,一汽,夏利n3,xiali-n3,110,NaN


In [46]:
temp = 
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1765 entries, 0 to 966
Data columns (total 7 columns):
brand_id                 1765 non-null int64
model_id                 1765 non-null int64
brand_name               1765 non-null object
model_name               1765 non-null object
model_slug               1765 non-null object
car_autohome_brand_id    1765 non-null int64
car_autohome_model_id    1325 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 110.3+ KB
